# Setup

In [11]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd
import copy

In [12]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [13]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [14]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [15]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j:
            querycounts[j['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            print(j)
            return j

In [16]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [17]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

In [18]:
def ac_to_table(aragorn_result,mnode):
    scores = []
    answer_node_count = []
    merged_count = []
    method = []
    extra = []
    for res_i, result in enumerate(aragorn_result['message']['results']):
        scores.append(result['score'])
        answer_node_count.append(len(result['node_bindings']))
        merged_count.append(len(result['node_bindings'][mnode]))
        try:
            method.append(result['node_bindings'][mnode][0]['coalescence_method'])
        except:
            method.append('Original')
    df = pd.DataFrame({'N_Answer_Nodes':answer_node_count, 'N_Merged_Nodes':merged_count, 'Method':method, 'Score':scores})
    return df

def filter_to_simple(aragorn_result,mnode):
    simple_result = copy.deepcopy(aragorn_result)
    simple_result['message']['results'] = list(
    filter( lambda x: 'coalescence_method' not in x['node_bindings'][mnode][0], 
           aragorn_result['message']['results'])
    )
    print(len(simple_result['message']['results']))
    return simple_result

def print_nodenames(simple_result,qnode):
    #Print the names of the answers
    for result in simple_result['message']['results']:
        #Each answer has an identifier:
        n1_id = result['node_bindings'][qnode][0]['id']
        #The information for that identifier is in the KG:
        node = simple_result['message']['knowledge_graph']['nodes'][n1_id]
        #Each node has a name
        print(node['name'])

def filter_to_coal(aragorn_result,mnode,method):
    #The results that have been coalesced:
    coalesced_results = list(
        filter( lambda x: 'coalescence_method'  in x['node_bindings'][mnode][0], 
               aragorn_result['message']['results'])
    )
    #Those that have been coalesced via a new node (graph coalescence)
    graph_coalesced_results = list(
        filter( lambda x: x['node_bindings'][mnode][0]['coalescence_method'] == method, coalesced_results)
    )
    print(len(graph_coalesced_results))
    simple_result = copy.deepcopy(aragorn_result)
    simple_result['message']['results'] = graph_coalesced_results
    return simple_result

def filter_to_gc(aragorn_result,mnode):
    return filter_to_coal(aragorn_result,mnode,'graph_enrichment')

def filter_to_pc(aragorn_result,mnode):
    return filter_to_coal(aragorn_result,mnode,'property_enrichment')

In [19]:
def submit_to_ars(m):
    submit_url = 'https://ars.transltr.io/ars/api/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'https://arax.ncats.io/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        if child['status'] == 'Done':
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [11]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "UMLS:C0221347",
                    "category": "biolink:PhenotypicFeature"
                },
                "n1": {
                    "category": "biolink:NamedThing"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## Strider Direct

In [14]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 0 results in 0:04:59.169225.


### Provenance

In [15]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count


### Queried sources

In [16]:
print_queried_sources(strider_result)

https://api.bte.ncats.io/v1/query (6 times)
https://translator.broadinstitute.org/genetics_provider/trapi/v1.0/query (2 times)
https://automat.renci.org/uberongraph/reasonerapi (85 times)
https://automat.renci.org/cord19-scibite/reasonerapi (18 times)
https://stars-app.renci.org/sparql-kp/query (57 times)
https://automat.renci.org/hetio/reasonerapi (5 times)
https://automat.renci.org/ontological-hierarchy/reasonerapi (8 times)
https://automat.renci.org/biolink/reasonerapi (11 times)
https://automat.renci.org/hmdb/reasonerapi (2 times)
https://automat.renci.org/ctd/reasonerapi (10 times)
https://translator.broadinstitute.org/molepro/trapi/v1.0/query (1 times)
https://automat.renci.org/covid-phenotypes/reasonerapi (3 times)
https://automat.renci.org/mychem/reasonerapi (5 times)
https://automat.renci.org/pharos/reasonerapi (2 times)
https://icees.renci.org:16340/knowledge_graph_one_hop?reasoner=true (2 times)


### Errors

In [17]:
print_errors(strider_result)

504 Server Error: Gateway Time-out for url: https://automat.renci.org/cord19-scigraph/reasonerapi (30 times)
429 Client Error: Too Many Requests for url: https://api.bte.ncats.io/v1/query (963 times)
502 Server Error: Bad Gateway for url: https://monarch-sandbox.cgrb.oregonstate.edu/query (47 times)
500 Server Error: Internal Server Error for url: https://automat.renci.org/uberongraph/reasonerapi (11 times)
500 Server Error: Internal Server Error for url: https://automat.renci.org/mychem/reasonerapi (1 times)
 (7 times)
500 Server Error: Internal Server Error for url: https://automat.renci.org/ctd/reasonerapi (1 times)
can't handle event type (90 times)
500 Server Error: Internal Server Error for url: http://chp.thayer.dartmouth.edu/query/ (2 times)


### Results

In [18]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

## ARAGORN 

In [19]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

Error, no result field


In [20]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

In [27]:
tq={
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "id": "HP:0033127",
                    "category": "biolink:PhenotypicFeature"
                },
                "n2": {
                    "category": "biolink:Disease"
                }
            },
            "edges": {
                "e02": {
                    "subject": "n1",
                    "object": "n2",
                    "predicate": "biolink:affected_by"
                }
            }
        }
    }
}

### ARAGORN Assessment

How did we do?